In [38]:
import os
import pandas as pd
import numpy as np
from scipy.optimize import minimize

## Global Params

In [27]:
model_name = 'simulation-hawkes'
n_sims = 10
inputs_path = os.path.join(os.getcwd(), 'data', 'inputs')
outputs_path = os.path.join(os.getcwd(), 'data', 'outputs')

## Load Simulation

In [31]:
ric = "fake"
dataPath = os.path.join(outputs_path, model_name)
dictBinnedData = {}
for d in range(n_sims):
    dictBinnedData[d] = []
dates = list(dictBinnedData.keys())

dfs = []
for i in dates:
    try:
        read_path = os.path.join(dataPath, ric + "_" + str(i) + "_12D.csv")
        df = pd.read_csv(read_path)

        # drop columns
        df.drop(['Unnamed: 0'], axis=1, inplace=True)

        # change columns order
        df = df[["Date", "Time", "event"] + df.drop(["Date", "Time", "event"], axis=1).columns.to_list()]

        dfs.append(df)
    except:
        print(f"No data for {ric} on {i}")
df = pd.concat(dfs)

df.tail()

Date        Time        event  Ask Price 1  Bid Price 1  Ask Price 2  \
347     9   98.942260  co_deep_Ask         45.0        44.99        45.02   
348     9   98.942339  co_deep_Ask         45.0        44.99        45.02   
349     9   98.942505  co_deep_Ask         45.0        44.99        45.02   
350     9   98.942836  co_deep_Ask         45.0        44.99        45.02   
351     9  106.865217  co_deep_Bid         45.0        44.99        45.02   

     Bid Price 2  BidDiff  AskDiff  BidDiff2  AskDiff2  
347        44.98      0.0      0.0       0.0       0.0  
348        44.98      0.0      0.0       0.0       0.0  
349        44.98      0.0      0.0       0.0       0.0  
350        44.98      0.0      0.0       0.0       0.0  
351        44.98      0.0      0.0       0.0       0.0

In [32]:
df["event"].unique()

array(['co_top_Ask', 'mo_Bid', 'lo_top_Bid', 'lo_inspread_Bid',
       'lo_inspread_Ask', 'co_deep_Bid', 'mo_Ask', 'lo_deep_Bid',
       'lo_top_Ask', 'co_top_Bid', 'co_deep_Ask', 'lo_deep_Ask'],
      dtype=object)

## Prepare Data

In [36]:
pivot_df = df.copy()
pivot_df["count"] = 1

pivot_df = pivot_df.pivot_table(index=["Date", "Time"], columns="event", values="count").fillna(0)

X = pivot_df.values

pivot_df.head()

event          co_deep_Ask  co_deep_Bid  co_top_Ask  co_top_Bid  lo_deep_Ask  \
Date Time                                                                      
0    0.151024          0.0          0.0         1.0         0.0          0.0   
     0.159023          0.0          0.0         0.0         0.0          0.0   
     0.159032          0.0          0.0         0.0         0.0          0.0   
     0.159052          0.0          0.0         0.0         0.0          0.0   
     0.159081          0.0          0.0         0.0         0.0          0.0   

event          lo_deep_Bid  lo_inspread_Ask  lo_inspread_Bid  lo_top_Ask  \
Date Time                                                                  
0    0.151024          0.0              0.0              0.0         0.0   
     0.159023          0.0              0.0              0.0         0.0   
     0.159032          0.0              0.0              0.0         0.0   
     0.159052          0.0              0.0              1.0         0.0   
     0.159081          0.0              0.0              1.0         0.0   

event          lo_top_Bid  mo_Ask  mo_Bid  
Date Time                                  
0    0.151024         0.0     0.0     0.0  
     0.159023         0.0     0.0     1.0  
     0.159032         1.0     0.0     0.0  
     0.159052         0.0     0.0     0.0  
     0.159081         0.0     0.0     0.0

In [37]:
X.shape

(3793, 12)

## Log-likelihood for a M-variate Hawkes Process with Exponential Excitation Kernels

### 1) Single observation


#### Intuitive Version

In [111]:
# model init params
np.random.seed(42)
M = X.shape[1]
T = X.shape[0]
ts = np.arange(0, T)

# model estimation params
mu = np.random.uniform(0.1, 1.0, M)
alpha = np.random.uniform(0.01, 0.5, (M, M))
beta = np.random.uniform(0.5, 2.0, (M, M))

# init recursive function
R = np.zeros((M, T, M))

for m in range(M):

    for n in range(M):

        tks = np.nonzero(X[:, m])[0]
        tis = np.nonzero(X[:, n])[0]
        for t in range(len(tks)):
            if t == 0:
                continue
            else:
                tk = tks[t]
                tkm1 = tks[t - 1]

                ti = tis[tis < tk]
                ti = ti[ti > tkm1]

                sum_over_tis_from_n = 0
                for s in range(len(ti)):
                    if s == 0:
                        continue
                    else:
                        sum_over_tis_from_n += np.exp(-beta[m, n] * (ti[s] - ti[s - 1]))

                R[m, t, n] = np.exp(-beta[m, n] * (tk - tkm1)) * R[m, t - 1, n] + sum_over_tis_from_n

log_likelihood = 0
for m in range(M):

     integral_term_over_t = 0
     for n in range(M):
          for t in ts:
               integral_term_over_t += (alpha[m, n] / beta[m, n] * (1 - np.exp(-beta[m, n] * (T - t))))
     integral_term_over_t = -(mu[m] * T) - integral_term_over_t

     integra_term_over_countp = 0
     for t in ts:
          for n in range(M):
               integra_term_over_countp += np.log(mu[n] + (alpha[m, n] * R[m, t, n]))

     log_likelihood += integral_term_over_t + integra_term_over_countp

In [112]:
log_likelihood

-566307.6579780579

#### Optimized Version

In [109]:
# model init params
np.random.seed(42)
M = X.shape[1]
T = X.shape[0]
ts = np.arange(0, T)

# model estimation params
mu = np.random.uniform(0.1, 1.0, M)
alpha = np.random.uniform(0.01, 0.5, (M, M))
beta = np.random.uniform(0.5, 2.0, (M, M))

# init recursive function
R = np.zeros((M, T, M))

log_likelihood = 0
for m in range(M):

     integral_term_over_t = 0
     for n in range(M):

          tks = np.nonzero(X[:, m])[0]
          tis = np.nonzero(X[:, n])[0]
          for t in range(len(tks)):
              if t == 0:
                  continue
              else:
                    tk = tks[t]
                    tkm1 = tks[t - 1]

                    ti = tis[tis < tk]
                    ti = ti[ti > tkm1]

                    sum_over_tis_from_n = 0
                    for s in range(len(ti)):
                         if s == 0:
                              continue
                         else:
                              sum_over_tis_from_n += np.exp(-beta[m, n] * (ti[s] - ti[s - 1]))

                    R[m, t, n] = np.exp(-beta[m, n] * (tk - tkm1)) * R[m, t - 1, n] + sum_over_tis_from_n

          for t in ts:
               integral_term_over_t += (alpha[m, n] / beta[m, n] * (1 - np.exp(-beta[m, n] * (T - t))))
     integral_term_over_t = -(mu[m] * T) - integral_term_over_t

     integra_term_over_countp = 0
     for t in ts:
          for n in range(M):
               integra_term_over_countp += np.log(mu[n] + (alpha[m, n] * R[m, t, n]))

     log_likelihood += integral_term_over_t + integra_term_over_countp

In [110]:
log_likelihood

-566307.6579780579

### 2) Full dataset

In [ ]:
def exponential_excitation_log_likelihood(T, M, event_times, mu, alpha, beta):
    """
    Computes the log-likelihood of an M-variate Hawkes process.

    Parameters:
    T (float): Observation window [0, T]
    M (int): Number of dimensions
    event_times (list of lists): Event times for each dimension, i.e., [[t1^1, t2^1, ...], [t1^2, t2^2, ...], ...]
    mu (np.array): Base intensity vector (shape: (M,))
    alpha (np.array): Excitation matrix (shape: (M, M))
    beta (np.array): Decay parameter matrix (shape: (M, M))

    Returns:
    float: Log-likelihood of a Hawkes process with exponential excitation kernels
    """

    # Initialize the log-likelihood
    log_likelihood = 0

    for m in range(M):
        # Compute the log-likelihood component for dimension m
        R = np.zeros((M, len(event_times[m]) + 1))
        integral_term = mu[m] * T
        summation_term = 0

        for k in range(len(event_times[m])):
            t_km = event_times[m][k]

            # Update R values recursively
            for n in range(M):
                if k > 0:
                    t_km_1 = event_times[m][k-1]
                    R[m, n] = np.exp(-beta[m, n] * (t_km - t_km_1)) * R[m, n]
                    R[m, n] += np.sum([np.exp(-beta[m, n] * (t_km - t_i)) for t_i in event_times[n] if t_km_1 < t_i < t_km])
                else:
                    R[m, n] = np.sum([np.exp(-beta[m, n] * (t_km - t_i)) for t_i in event_times[n] if t_i < t_km])

            # Update the intensity function lambda_m(t_km)
            lambda_m_k = mu[m] + np.sum([alpha[m, n] * R[m, n] for n in range(M)])

            # Add to summation term (log-likelihood component for observed events)
            summation_term += np.log(lambda_m_k)

            # Add to integral term (log-likelihood component for integrated intensity)
            integral_term += (alpha[m, n] / beta[m, n]) * (1 - np.exp(-beta[m, n] * (T - t_km)))

        log_likelihood += -integral_term + summation_term

    return log_likelihood

In [ ]:
def objective(params):
    mu = params[:M]
    alpha = params[M:M + M * M].reshape((M, M))
    beta = params[M + M * M:].reshape((M, M))
    return -exponential_excitation_log_likelihood(T, M, xt, mu, alpha, beta)

In [39]:
T = X.shape[0]
M = X.shape[1]

mu = np.random.uniform(0.1, 1.0, M)
alpha = np.random.uniform(0.01, 0.5, (M, M))
beta = np.random.uniform(0.5, 2.0, (M, M))